In [1]:
####
####  ---------------微调--------利用已有的训练好的模型，在小样本上微调。
####

import os 
print('directory:',os.getcwd())#知道路径，以免访问文件时候出错。

directory: /home/liuchen/jupyterprj/transferlearning


In [2]:
##导入base_model
from keras.applications.inception_v3 import InceptionV3
base_model=InceptionV3(weights='imagenet',include_top=False)

Using TensorFlow backend.


In [3]:
import sys
from keras.layers import GlobalAveragePooling2D,Dense
from keras.models import Model
x=base_model.output
x=GlobalAveragePooling2D()(x)
x=Dense(1024,activation='relu')(x)
predictions=Dense(5,activation='softmax')(x)
model=Model(inputs=base_model.input,output=predictions)

Instructions for updating:
keep_dims is deprecated, use keepdims instead


/home/liuchen/anaconda3/envs/python3/lib/python3.6/site-packages/ipykernel_launcher.py:8: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("de...)`
  


In [7]:
from keras.optimizers import Adagrad
def setup_to_fine_tune(model,base_model):
    GAP_LAYER=17
    for layer in base_model.layers[:GAP_LAYER+1]:
        layer.trainable=False
    for layer in base_model.layers[GAP_LAYER+1:]:
        layer.trainable=True
    model.compile(optimizer=Adagrad(lr=0.0001),loss='categorical_crossentropy',metrics=['accuracy'])
setup_to_fine_tune(model=model,base_model=base_model)

In [8]:
##########数据准备
####处理数据
from keras.preprocessing.image import ImageDataGenerator
train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
        './re/train', 
        target_size=(150, 150),  # all images will be resized to 150x150
        batch_size=32,
        class_mode='categorical')                               # matt，多分类

validation_generator = test_datagen.flow_from_directory(
        './re/test',
        target_size=(150, 150),
        batch_size=32,
        class_mode='categorical') 

Found 400 images belonging to 5 classes.
Found 100 images belonging to 5 classes.


In [ ]:
###########训练模型
history_ft = model.fit_generator(generator=train_generator,
                                 steps_per_epoch=10, 
                                 epochs=2, 
                                 validation_data=validation_generator, 
                                 validation_steps=5, 
                                 class_weight='auto')


# history_ft = model.fit_generator(train_generator,
#     nb_epoch=1,
#     samples_per_epoch=50,
#     validation_data=validation_generator,
#     nb_val_samples=50,
#     class_weight='auto')
# model.save('./flowers17_iv3_ft.h5')


Epoch 1/2
10/10 [==============================] - 38s - loss: 0.0799 - acc: 0.9969 - val_loss: 0.0027 - val_acc: 1.0000
Epoch 2/2
 8/10 [=======================>......] - ETA: 6s - loss: 0.0486 - acc: 0.9961 